## Introducción

En esta lección se cubrirá:
- Qué es la llamada a funciones y en qué casos se utiliza
- Cómo crear una llamada a función usando Azure OpenAI
- Cómo integrar una llamada a función en una aplicación

## Objetivos de aprendizaje

Después de completar esta lección, sabrás cómo y entenderás:

- El propósito de utilizar la llamada a funciones
- Configurar la llamada a funciones usando el servicio Azure OpenAI
- Diseñar llamadas a funciones efectivas para el caso de uso de tu aplicación


## Entendiendo las llamadas a funciones

En esta lección, queremos crear una función para nuestra startup educativa que permita a los usuarios usar un chatbot para encontrar cursos técnicos. Recomendaremos cursos que se ajusten a su nivel de habilidad, rol actual y tecnología de interés.

Para lograr esto, usaremos una combinación de:
 - `Azure Open AI` para crear una experiencia de chat para el usuario
 - `Microsoft Learn Catalog API` para ayudar a los usuarios a encontrar cursos según su solicitud
 - `Function Calling` para tomar la consulta del usuario y enviarla a una función que realice la solicitud a la API.

Para empezar, veamos por qué querríamos usar llamadas a funciones en primer lugar:

print("Mensajes en la siguiente solicitud:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # obtener una nueva respuesta de GPT donde puede ver la respuesta de la función


print(second_response.choices[0].message)


### Por qué usar Function Calling

Si ya completaste alguna otra lección de este curso, probablemente ya entiendes el poder de usar Modelos de Lenguaje Grande (LLMs). Esperamos que también puedas notar algunas de sus limitaciones.

Function Calling es una función del Azure Open AI Service que ayuda a superar las siguientes limitaciones:
1) Formato de respuesta consistente
2) Capacidad de usar datos de otras fuentes de una aplicación en un contexto de chat

Antes de function calling, las respuestas de un LLM eran no estructuradas e inconsistentes. Los desarrolladores tenían que escribir código de validación complejo para poder manejar cada variación de una respuesta.

Los usuarios no podían obtener respuestas como "¿Cuál es el clima actual en Estocolmo?". Esto se debe a que los modelos estaban limitados a la época en la que se entrenaron los datos.

Veamos el siguiente ejemplo que ilustra este problema:

Supongamos que queremos crear una base de datos de datos de estudiantes para poder sugerirles el curso adecuado. A continuación tenemos dos descripciones de estudiantes que son muy similares en los datos que contienen.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

Queremos enviar esto a un LLM para analizar los datos. Esto se puede usar más adelante en nuestra aplicación para enviarlo a una API o almacenarlo en una base de datos.

Vamos a crear dos prompts idénticos en los que le indicamos al LLM qué información nos interesa:


Queremos enviar esto a un LLM para analizar las partes que son importantes para nuestro producto. Así podemos crear dos indicaciones idénticas para instruir al LLM:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


Después de crear estos dos prompts, los enviaremos al LLM usando `openai.ChatCompletion`. Almacenamos el prompt en la variable `messages` y asignamos el rol a `user`. Esto es para imitar un mensaje de un usuario escrito a un chatbot.


In [ ]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-07-01-preview"
  )

deployment=os.environ['AZURE_OPENAI_DEPLOYMENT']

: 

In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

Aunque los prompts sean los mismos y las descripciones parecidas, podemos obtener diferentes formatos para la propiedad `Grades`.

Si ejecutas la celda anterior varias veces, el formato puede ser `3.7` o `3.7 GPA`.

Esto ocurre porque el LLM toma datos no estructurados en forma de prompt escrito y también devuelve datos no estructurados. Necesitamos tener un formato estructurado para saber qué esperar al almacenar o usar estos datos.

Al usar llamadas funcionales, podemos asegurarnos de recibir datos estructurados. Cuando usamos llamadas a funciones, el LLM en realidad no llama ni ejecuta ninguna función. En su lugar, creamos una estructura que el LLM debe seguir en sus respuestas. Luego usamos esas respuestas estructuradas para saber qué función ejecutar en nuestras aplicaciones.


![Diagrama de flujo de llamada de función](../../../../translated_images/Function-Flow.083875364af4f4bb69bd6f6ed94096a836453183a71cf22388f50310ad6404de.es.png)


### Casos de uso para llamadas a funciones

**Llamar herramientas externas**
Los chatbots son muy útiles para responder preguntas de los usuarios. Al usar llamadas a funciones, los chatbots pueden aprovechar los mensajes de los usuarios para realizar ciertas tareas. Por ejemplo, un estudiante puede pedirle al chatbot: "Envía un correo a mi profesor diciendo que necesito más ayuda con esta materia". Esto puede hacer una llamada a la función `send_email(to: string, body: string)`

**Crear consultas a API o bases de datos**
Los usuarios pueden buscar información usando lenguaje natural que se convierte en una consulta o solicitud de API con formato. Un ejemplo podría ser un profesor que pregunta: "¿Quiénes son los estudiantes que completaron la última tarea?", lo que podría llamar a una función llamada `get_completed(student_name: string, assignment: int, current_status: string)`

**Crear datos estructurados**
Los usuarios pueden tomar un bloque de texto o un archivo CSV y usar el LLM para extraer información relevante. Por ejemplo, un estudiante puede convertir un artículo de Wikipedia sobre acuerdos de paz para crear tarjetas de estudio con IA. Esto se puede lograr usando una función llamada `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. Creando tu primera llamada a una función

El proceso para crear una llamada a una función incluye 3 pasos principales:
1. Llamar a la API de Chat Completions con una lista de tus funciones y un mensaje del usuario
2. Leer la respuesta del modelo para realizar una acción, es decir, ejecutar una función o una llamada a una API
3. Hacer otra llamada a la API de Chat Completions con la respuesta de tu función para usar esa información y crear una respuesta para el usuario.


![Flujo de una llamada a función](../../../../translated_images/LLM-Flow.3285ed8caf4796d7343c02927f52c9d32df59e790f6e440568e2e951f6ffa5fd.es.png)


### Elementos de una llamada a función

#### Entrada del usuario

El primer paso es crear un mensaje de usuario. Este puede asignarse dinámicamente tomando el valor de una entrada de texto, o puedes asignar un valor aquí. Si es la primera vez que trabajas con la API de Chat Completions, necesitamos definir el `role` y el `content` del mensaje.

El `role` puede ser `system` (creando reglas), `assistant` (el modelo) o `user` (el usuario final). Para la llamada a función, lo asignaremos como `user` y una pregunta de ejemplo.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### Creando funciones.

A continuación definiremos una función y los parámetros de esa función. Aquí usaremos solo una función llamada `search_courses`, pero puedes crear varias funciones.

**Importante**: Las funciones se incluyen en el mensaje del sistema para el LLM y contarán dentro de la cantidad de tokens disponibles que tienes.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**Definiciones**

`name` - El nombre de la función que queremos que se llame.

`description` - Esta es la descripción de cómo funciona la función. Aquí es importante ser específico y claro.

`parameters` - Una lista de valores y el formato que quieres que el modelo genere en su respuesta.

`type` - El tipo de dato en el que se almacenarán las propiedades.

`properties` - Lista de los valores específicos que el modelo usará para su respuesta.

`name` - El nombre de la propiedad que el modelo usará en su respuesta formateada.

`type` - El tipo de dato de esta propiedad.

`description` - Descripción de la propiedad específica.

**Opcional**

`required` - Propiedad obligatoria para que se complete la llamada a la función.


### Realizar la llamada a la función
Después de definir una función, ahora necesitamos incluirla en la llamada a la API de Chat Completion. Hacemos esto agregando `functions` a la solicitud. En este caso, `functions=functions`.

También existe la opción de establecer `function_call` en `auto`. Esto significa que dejaremos que el LLM decida qué función debe llamarse según el mensaje del usuario en lugar de asignarla nosotros mismos.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

Ahora veamos la respuesta y observemos cómo está formateada:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

Puedes ver que se llama al nombre de la función y, a partir del mensaje del usuario, el LLM pudo encontrar los datos para completar los argumentos de la función.


## 3. Integrando llamadas a funciones en una aplicación.

Después de haber probado la respuesta formateada del LLM, ahora podemos integrarla en una aplicación.

### Gestionando el flujo

Para integrar esto en nuestra aplicación, sigamos los siguientes pasos:

Primero, hagamos la llamada a los servicios de Open AI y guardemos el mensaje en una variable llamada `response_message`.


In [ ]:
response_message = response.choices[0].message

Ahora definiremos la función que llamará a la API de Microsoft Learn para obtener una lista de cursos:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



Como buena práctica, veremos si el modelo quiere llamar a una función. Después de eso, crearemos una de las funciones disponibles y la asignaremos a la función que se está llamando.
Luego tomaremos los argumentos de la función y los relacionaremos con los argumentos provenientes del LLM.

Por último, agregaremos el mensaje de llamada a la función y los valores que fueron devueltos por el mensaje `search_courses`. Esto le da al LLM toda la información que necesita para
responder al usuario usando lenguaje natural.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## Desafío de Código

¡Buen trabajo! Para seguir aprendiendo sobre Azure Open AI Function Calling puedes construir: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst
 - Más parámetros de la función que puedan ayudar a los estudiantes a encontrar más cursos. Puedes encontrar los parámetros disponibles de la API aquí:
 - Crear otra llamada de función que tome más información del estudiante, como su idioma nativo
 - Crear manejo de errores cuando la llamada de función y/o la llamada a la API no devuelvan cursos adecuados



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Si bien nos esforzamos por lograr precisión, tenga en cuenta que las traducciones automáticas pueden contener errores o inexactitudes. El documento original en su idioma nativo debe considerarse la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de ningún malentendido o interpretación errónea que surja del uso de esta traducción.
